# GPT-3.5 Turbo: Cluster Category Name Generator
This notebook uses OpenAI's GPT-3.5 Turbo to automatically suggest concise, non-generic names for product clusters.
It uses the product names from each cluster (generated by the clustering model) and generates a human-readable category label.

### Step 1: Import Libraries and Load Environment Variables
We use the `openai` API and `dotenv` to securely load your API key from a `.env` file.

### Step 2: Define the Name Generator Function
This function sends a list of product names to the OpenAI GPT-3.5 Turbo API to get a meaningful category name.
It avoids reusing names by checking against already used names.

### Step 3: Generate Category Names per Cluster
For each cluster from the previous model, we call the function and map each cluster ID to its corresponding name.
The final result is stored in the column `clustered_category`.

In [ ]:
# !pip install dotenv
import openai
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

def get_category_name_openai(product_names, openai_api_key, model="gpt-3.5-turbo"):
    """
    Given a list of product names, use OpenAI to suggest a concise, precise unique category name.
    Returns only the category name as a string.
    """
    prompt = (
        "Given the following list of product names, suggest a concise, precise category name that best describes the majority of products and it is not generic. "
        f"Do NOT use any of these words: {', '.join(used_names)}\n"
        "Only return the category name, nothing else.\n\n"
        "Product names:\n" +
        "\n".join(product_names)
    )
    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=15,
    )
    return response.choices[0].message.content.strip()

# After clustering, replace your static cluster_name_map with OpenAI-generated names:
cluster_name_map = {}
used_names = set()

# Generate unique category names for each cluster using OpenAI
for cluster_id in sorted(df['cluster'].unique()):
    product_names = df[df['cluster'] == cluster_id]['name'].dropna().unique().tolist()
    if product_names:
        category_name = get_category_name_openai(product_names, openai_api_key)
        while category_name in used_names:
            category_name = get_category_name_openai(product_names, openai_api_key)
        used_names.add(category_name)
        cluster_name_map[cluster_id] = category_name

print(cluster_name_map)

df['clustered_category'] = df['cluster'].map(cluster_name_map)
